In [1]:
# Import packages

import pandas as pd 
import numpy as np 
import os

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

import statsmodels.api as sm
from scipy import stats

from joblib import dump, load

pd.set_option('display.max_columns', None)

In [2]:
nfl = pd.read_csv('../../data/value_models_combined.csv')
nfl_rolling = pd.read_csv('../../data/value_models_combined_6_game_rolling.csv')

In [3]:
nfl

,Unnamed: 0,season,week,team,opponent,score,opponent_score,qb,qb_value,passing_value,rushing_value,qb_rushing_value_pct,qb_def_value,pass_def_value,rush_def_value,special_teams_value,team_full,opponent_full
0,0,2014,1,ARI,LAC,18,17,C. Palmer,0.491486,0.472651,-1.258735,0,0.361331,0.014745,1.468394,-0.996743,Arizona Cardinals,Los Angeles Chargers
1,1,2014,1,ATL,NO,37,34,M. Ryan,1.997776,1.515659,0.682493,0,-1.047285,-0.748551,-1.005162,-0.243872,Atlanta Falcons,New Orleans Saints
2,2,2014,1,BAL,CIN,16,23,J. Flacco,-0.509634,-0.552992,0.180459,0,-0.457796,-0.934747,0.727127,-0.442100,Baltimore Ravens,Cincinnati Bengals
3,3,2014,1,BUF,CHI,23,20,E. Manuel,-0.171272,-0.063069,1.014793,0,0.198708,0.700995,-0.187514,0.780553,Buffalo Bills,Chicago Bears
4,4,2014,1,CAR,TB,20,14,D. Anderson,0.728679,0.690009,-0.984362,0,0.531960,1.035469,-0.392165,-0.725290,Carolina Panthers,Tampa Bay Buccaneers
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4866,4866,2022,21,KC,CIN,23,20,P. Mahomes,0.500521,0.593604,-0.632606,0,0.054898,0.253622,-0.804881,-0.286208,Kansas City Chiefs,Cincinnati Bengals
4867,4867,2022,21,PHI,SF,31,7,J. Hurts,-0.148775,-0.158965,1.337421,0,1.277398,0.753996,2.334398,-0.597381,Philadelphia Eagles,San Francisco 49ers
4868,4868,2022,21,SF,PHI,7,31,J. Johnson,-0.835878,-0.286917,-2.334398,0,0.128784,0.147045,-1.337421,0.597381,San Francisco 49ers,Philadelphia Eagles
4869,4869,2022,22,KC,PHI,38,35,P. Mahomes,1.243585,1.060283,1.471309,0,-1.161197,-1.083680,0.026369,-0.277841,Kansas City Chiefs,Philadelphia Eagles


In [4]:
qb_def = nfl[['season', 'team', 'opponent', 'week', 'team_full', 'opponent_full', 'score', 'opponent_score',
          'qb_def_value']]
opposing_off = nfl_rolling[['season', 'week', 'team_full', 
          'qb_value']]

In [5]:
df = qb_def.merge(opposing_off, left_on=['season', 'week', 'opponent_full'], 
             right_on=['season', 'week', 'team_full'], suffixes = ('','_opponent'))
df = df.drop(columns=['team_full_opponent'])

In [6]:
def value_adjustment(value, adjusting_value, original_value_percentage=.95, adjustment_threshold=.1):
    value_over_expected = value + adjusting_value
    adjusted_value = (value_over_expected*adjustment_threshold) + (value*original_value_percentage)
    adjustment = adjusted_value - value
    
    return adjustment, adjusted_value

df[['qb_def_adjustment', 'qb_def_adjusted_value']] = df.apply(lambda x: value_adjustment(x.qb_def_value, x.qb_value, 
                                original_value_percentage=.7, adjustment_threshold=.3), axis=1, result_type='expand')

In [7]:
df.to_csv('qb_defense_value_modeling_adjusted_data/qb_defense_values_with_adjustment.csv')

In [8]:
testing = df.dropna()
testing.sort_values(by='qb_def_adjustment', ascending=False)

,season,team,opponent,week,team_full,opponent_full,score,opponent_score,qb_def_value,qb_value,qb_def_adjustment,qb_def_adjusted_value
3774,2021,JAX,DEN,2,Jacksonville Jaguars,Denver Broncos,13,23,-0.730503,2.030331,0.609099,-0.121403
38,2014,CIN,ATL,2,Cincinnati Bengals,Atlanta Falcons,24,10,1.400317,1.997776,0.599333,1.999650
36,2014,CAR,DET,2,Carolina Panthers,Detroit Lions,24,7,0.149692,1.934444,0.580333,0.730025
2700,2019,DAL,WAS,2,Dallas Cowboys,Washington Football Team,31,21,-0.423989,1.812050,0.543615,0.119626
1014,2015,ARI,SEA,17,Arizona Cardinals,Seattle Seahawks,6,36,-1.212171,1.742944,0.522883,-0.689287
...,...,...,...,...,...,...,...,...,...,...,...,...
558,2015,NYJ,CLE,1,New York Jets,Cleveland Browns,31,10,0.767077,-1.991945,-0.597584,0.169493
3006,2019,PIT,CIN,12,Pittsburgh Steelers,Cincinnati Bengals,16,10,0.981831,-2.148775,-0.644632,0.337199
2360,2018,NE,BUF,8,New England Patriots,Buffalo Bills,25,6,1.678389,-2.326150,-0.697845,0.980544
2973,2019,LV,CIN,11,Las Vegas Raiders,Cincinnati Bengals,17,10,1.832384,-2.403068,-0.720920,1.111463


In [9]:
df[(df.team=='ATL') & (df.season==2014)]

,season,team,opponent,week,team_full,opponent_full,score,opponent_score,qb_def_value,qb_value,qb_def_adjustment,qb_def_adjusted_value
1,2014,ATL,NO,1,Atlanta Falcons,New Orleans Saints,37,34,-1.047285,NaN,NaN,NaN
33,2014,ATL,CIN,2,Atlanta Falcons,Cincinnati Bengals,10,24,-1.014977,0.452303,0.135691,-0.879286
65,2014,ATL,TB,3,Atlanta Falcons,Tampa Bay Buccaneers,56,14,1.030735,NaN,NaN,NaN
96,2014,ATL,MIN,4,Atlanta Falcons,Minnesota Vikings,28,41,-1.174822,0.283517,0.085055,-1.089767
123,2014,ATL,NYG,5,Atlanta Falcons,New York Giants,20,30,-0.462420,0.195835,0.058751,-0.403669
153,2014,ATL,CHI,6,Atlanta Falcons,Chicago Bears,13,27,-1.230142,-0.079574,-0.023872,-1.254014
183,2014,ATL,BAL,7,Atlanta Falcons,Baltimore Ravens,7,29,-0.742118,0.495278,0.148583,-0.593535
213,2014,ATL,DET,8,Atlanta Falcons,Detroit Lions,21,22,-0.705817,-0.309834,-0.092950,-0.798767
269,2014,ATL,TB,10,Atlanta Falcons,Tampa Bay Buccaneers,27,17,-0.529290,-0.132286,-0.039686,-0.568976
295,2014,ATL,CAR,11,Atlanta Falcons,Carolina Panthers,19,17,0.469456,-0.628535,-0.188560,0.280896


In [10]:
qb_def[(qb_def.team=='ATL') & (qb_def.season==2014)]

,season,team,opponent,week,team_full,opponent_full,score,opponent_score,qb_def_value
1,2014,ATL,NO,1,Atlanta Falcons,New Orleans Saints,37,34,-1.047285
33,2014,ATL,CIN,2,Atlanta Falcons,Cincinnati Bengals,10,24,-1.014977
65,2014,ATL,TB,3,Atlanta Falcons,Tampa Bay Buccaneers,56,14,1.030735
96,2014,ATL,MIN,4,Atlanta Falcons,Minnesota Vikings,28,41,-1.174822
123,2014,ATL,NYG,5,Atlanta Falcons,New York Giants,20,30,-0.462420
153,2014,ATL,CHI,6,Atlanta Falcons,Chicago Bears,13,27,-1.230142
183,2014,ATL,BAL,7,Atlanta Falcons,Baltimore Ravens,7,29,-0.742118
213,2014,ATL,DET,8,Atlanta Falcons,Detroit Lions,21,22,-0.705817
269,2014,ATL,TB,10,Atlanta Falcons,Tampa Bay Buccaneers,27,17,-0.529290
295,2014,ATL,CAR,11,Atlanta Falcons,Carolina Panthers,19,17,0.469456


In [11]:
opposing_off[(opposing_off.team_full=='Tampa Bay Buccaneers') & (opposing_off.season==2014)]

,season,week,team_full,qb_value
29,2014,1,Tampa Bay Buccaneers,NaN
61,2014,2,Tampa Bay Buccaneers,-0.561916
93,2014,3,Tampa Bay Buccaneers,NaN
119,2014,4,Tampa Bay Buccaneers,-0.054512
149,2014,5,Tampa Bay Buccaneers,0.141839
179,2014,6,Tampa Bay Buccaneers,0.314280
239,2014,8,Tampa Bay Buccaneers,-0.005079
266,2014,9,Tampa Bay Buccaneers,-0.117971
292,2014,10,Tampa Bay Buccaneers,-0.132286
319,2014,11,Tampa Bay Buccaneers,0.086997
